__качаем объявления о продаже квартир с циан.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
import sys
import re
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep
from random import random

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

In [3]:
raw_data_path = 'data/raw/flat/cian' # каталог с файлами объявлений

In [4]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'
# base_url, req_param = 'https://sevastopol.cian.ru/kupit-kvartiru/', ''
place = 'sevastopol'
base_url, req_param ='https://www.cian.ru/cat.php', 'deal_type=sale&offer_type=flat&region=184723'

In [5]:
%%time

# from lib.downloader import DownloaderSimple
# from lib.parser import AdsListParser
from lib.downloader import DownloaderSeleniumFirefox
from lib.cian import CianParser


df,html = CianParser(
        base_url = base_url,
        driver=DownloaderSeleniumFirefox(profile_path),
        # driver=DownloaderSimple(),
    ).load(req_param=req_param, keep_html=True) # , npage_start=53) # , page_limit=3 ) #

print(len(df))

[INFO    ] 2023-01-27 16:27:20 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2023-01-27 16:27:20 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2023-01-27 16:27:23 | AdsListParser: downloader init
[INFO    ] 2023-01-27 16:27:23 | CianParser: downloader init
[INFO    ] 2023-01-27 16:27:23 | AdsListParser: start read and parse pages...
[INFO    ] 2023-01-27 16:27:41 | AdsListParser: read page 1
[INFO    ] 2023-01-27 16:27:49 | AdsListParser: read page 2
[INFO    ] 2023-01-27 16:28:04 | AdsListParser: read page 3
[INFO    ] 2023-01-27 16:28:13 | AdsListParser: read page 4
[INFO    ] 2023-01-27 16:28:25 | AdsListParser: read page 5
[INFO    ] 2023-01-27 16:28:33 | AdsListParser: read page 6
[INFO    ] 2023-01-27 16:28:41 | AdsListParser: read page 7
[INFO    ] 2023-01-27 16:28:51 | AdsListParser: read page 8
[INFO    ] 2023-01-27 16:29:00 | AdsListParser: read page 9
[INFO    ] 2023-01-27 16:29:11 | AdsListParser: read page 10
[INFO    ] 2023-01-27 16:29:17 | AdsLis

In [6]:
df.sample(3)

,OfferTitle,OfferSubtitle,Deadline,MainPrice,PriceInfo,GeoLabel,TimeLabel,LinkArea,Description,page,ts
640,"3-комн. кв., 40 м², 9/11 этаж",,дом сдан,10 000 000 ₽,250 000 ₽/м²,"[Севастополь, р-н Гагаринский, улица Парковая, 7]","[11 часов назад, сегодня, 04:56]",https://sevastopol.cian.ru/sale/flat/282200138/,"Продаётся 3-комнатная квартира в сданном доме (Корпус 1), срок сдачи: IV-кв. 2018, общей площадью 40 кв.м., на 9 этаже. Комплекс апартаментов ""Аква DeLuxe"" предназначены для комфортабельной жизни и отдыха прямо на берегу Черного моря! В курортный комплекс входят: рестораны, аквапарк, бассейны, банный комплекс, спа-центр, собственный пляж, детские площадки, красивый парк с уютными беседками, теннисные корты, спортивные площадки.",23,2023-01-27 16:37:11.689592
549,1-к в историческом центре города!,"1-комн. кв., 41,5 м², 3/4 этаж",,6 200 000 ₽,149 398 ₽/м²,"[Севастополь, р-н Ленинский, улица Пушкина, 12]","[3 недели назад, 3 янв, 15:11]",https://sevastopol.cian.ru/sale/flat/276640867/,"Продам хорошую 1 комнатную квартиру в историческом центре города Севастополя\nРасположена по улице ПУШКИНА, 12 (рядом ул. Ленина, Мичурина, Мокроусова, Большая Морская, Советская, пл. Суворова, Ушакова, Нахимова и др.)\n3 этаж в 4-х этажном доме\nБлагодаря толстым стенам в квартире зимой тепло, а летом очень комфортно и не жарко\nОбщая площадь 41,5м2: просторная комната 18м2, кухня 9м2, совмещенный санузел 4,9м2, застекленный балкон, с которого видно море!\nСостояние хорошее\nНа горячую воду установлена газовая колонка очень экономно по оплате коммунальных платежей\nОгромным плюсом для центра будет наличие парковки прямо во дворе\nВ пешей доступности расположены лучшие школы города: 3, 1, детский садик, а также все социально значимые объекты инфраструктуры\nИ, несомненно, изюминкой данного места являются большое количество достопримечательностей города, парка Исторический бульвар, скверов для прогулок и набережной Артбухта\nДанное предложение может быть интересно как для собственного проживания, так и прекрасная инвестиция для сдачи в аренду, ведь центр города всегда в спросе!\nЗвоните, записывайтесь на просмотр",20,2023-01-27 16:37:11.689592
1269,"3-комн. кв., 98 м², 1/5 этаж",,,12 500 000 ₽,127 551 ₽/м²,"[Севастополь, р-н Гагаринский, улица Героев Бреста, 46В]","[три дня назад, 24 янв, 12:21]",https://sevastopol.cian.ru/sale/flat/273487750/,"Четырехкомнатная квартира 98 м2, на 1/5 этаже дома с просторной кухней 13,68 км2. Расположена в г. Севастополь, Гагаринский район, ул. Героев Бреста, 46 к. В.\nПозвоните нам, чтобы узнать все детали! \nТехнические характеристики: \n \n-Адрес: г. Севастополь, Гагаринский район, ул. Героев Бреста, 46 к. В\n-Год постройки дома: 2002\n-Этаж/этажность: 1/5\n-Количество комнат: 4\n-Комнаты: изолированные\n-Общая площадь: 98 м2\n-Жилая площадь: 68,74 м2\n-Площадь кухни: 13,68 м2\n-Высота потолка: 2,6 м\n-Санузел: совмещенный\n-Водоснабжение: центральное\n-Горячее водоснабжение: центральное\n-Отопление: центральное\nДополнительная информация: \nКомнаты правильной формы позволят Вам воплотить в жизнь любое дизайнерское решение с грамотным использованием пространства.\nПозвоните нам, чтобы назначить просмотр! \nИнфраструктура: \nКвартира находится в тихом, спальном районе. На придомовой территории есть детская площадка. В 5 минутах ходьбы находятся сквер и торговый центр ПУД, в 10 минутах ТЦ Апельсин. До остановок общественного транспорта 5 минут ходьбы, откуда Вы сможете добраться в любую точку города. Так же, в 10 минутах ходьбы находятся школы 32, 49, 54, детские сады 92, 133, 130, отделения почты, небольшие продовольственные магазины, аптеки, поликлиника 3. До общественного пляжа 10 минут езды.\nПодходит для наличного расчета, под гражданскую ипотеку, материнский капитал, жилищные сертификаты. \nДокументы РФ проверены юристами и готовы к сделке. \nПрофессиональное сопровождение до получения права собственности.\nДобавьте предложение в закладки, чтобы не потерять!",46,2023-01-27 16:37:11.689592


In [7]:
assert len(df)>0

In [8]:
print(len(df))
df = pd.DataFrame(df).dropna().drop_duplicates(['LinkArea','Description',]).reset_index(drop=True)
print(len(df))

1500
1467


In [9]:
df['place']=place

In [10]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
# df.to_excel(f'{raw_data_path}/cian_{ts}_raw.xlsx',index=False)
df.to_pickle(f'{raw_data_path}/cian_{ts}_raw.pkl')

----

In [11]:
# df = pd.read_excel('data/raw/cian/cian_2022-09-08_13-49_raw.xlsx')

In [12]:
# df

In [13]:
# df['GeoLabel'].apply( lambda s : len(s) ) #', '.join(s) )

# s = ['Севастополь', 'р-н Гагаринский', 'Казачья Бухта мкр', 'Скифия ЖК']
# ', '.join(s)

In [14]:
# from lib.cian import CianDataCleaner

# CianDataCleaner().transform(df)

----

In [15]:
# with open('tmp/cian.html','wt') as f: f.write(html[0])